### 코로나로 인한 사람들의 흥미 변화 및 유망 품목을 파악하기 위해 앱 데이터를 크롤링해서 분석해보았습니다.

* 앱 순위를 크롤링한 사이트는 [모바일 인덱스](https://www.mobileindex.com/) 입니다.

먼저, 기간별로 어플 순위를 가져왔습니다.


이때, **구글 플레이 스토어(안드로이드)**와 **앱 스토어(IOS)**를 구분지어 데이터를 크롤링했습니다.  

**기간**은 **2019년 1월 ~ 12월**과 **2020년 1월 ~ 6월**의 데이터를 가져왔습니다.

### 기간별 앱 순위와 장르 가져오기

* 설정 초기화 및 필요한 모듈 임포트

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!apt install chromium-chromedriver
!pip install selenium

from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

import warnings 
warnings.filterwarnings('ignore')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (83.0.4103.61-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import plotly.express as px
import numpy as np

In [ ]:
genre_dic = {}

def makeUrl(dateList,i, kinds):
  if kinds == 'playstore': 
    url = 'https://www.mobileindex.com/app/get_rank_all?rt=r&mk=2&c=kr&t=app&rs=100&d=' + dateList[i]
  else:
    url = 'https://www.mobileindex.com/app/get_rank_all?rt=r&mk=1&c=kr&t=app&rs=100&d=' + dateList[i]
  return url

def getGenreUrl(url):
  driver = webdriver.Chrome('chromedriver',options=options)
  driver.get(url)

  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')
  genreURL = soup.select('div.item-info > a')
  return genreURL

def get_genre(url,appname):
  if appname in genre_dic:
    return genre_dic[appname]
  dr = webdriver.Chrome('chromedriver',options=options)
  dr.get(url)
  ht = dr.page_source
  so = BeautifulSoup(ht, 'html.parser')
  genre = so.select('tr.text-center')[0].text
  index1 = genre.find('\n',1)
  if genre[index1+1] == '\n':
    genre_dic[appname] = '기타'
    return '기타'
  index2 = genre.find('\n',14)
  genre_dic[appname] = genre[(index1+1):index2]
  return genre[(index1+1):index2]
  
def separateAppRank(rank):
  free_rank = []
  pay_rank = []
  sales_rank = [] 
  for j in range(len(rank)):
    if len(rank[j]) == 100:
      free_rank.append(rank[j][0::2])
      pay_rank.append(rank[j][1::2])
    elif len(rank[j]) == 150 or len(rank[j]) == 300:
      free_rank.append(rank[j][0::3])
      pay_rank.append(rank[j][1::3])
      sales_rank.append(rank[j][2::3])
  return free_rank, pay_rank, sales_rank

def getAppRank(url, year):
  driver = webdriver.Chrome('chromedriver',options=options)
  driver.get(url)

  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')
  notices = soup.select('span.appname')
  
  tmp = []
  for n in notices:
    tmp.append(n.text.strip())
  return tmp

In [ ]:
#2019
appRank_p = []
appRank_a = []
date_2019 = ['2019-01-31','2019-02-28','2019-03-31','2019-04-30','2019-05-31','2019-06-30','2019-07-31','2019-08-31','2019-09-30','2019-10-31','2019-11-30','2019-12-31']
date_size = len(date_2019)
kinds = ['playstore','appstore']

for kind in kinds:
  appRank = []
  appGenre = []
  for i in range(date_size):
    url = makeUrl(date_2019,i,kind)
    tmp = getAppRank(url,2019)
    appRank.append(tmp)
    genreURL = getGenreUrl(url)
    genreList = list()
    for j in range(len(genreURL)):
      genreList.append(get_genre('https://www.mobileindex.com/'+genreURL[j]['href'],tmp[j]))
    appGenre.append(genreList)
    print(kind,' data of ',date_2019[i],' is done')
  if kind == 'playstore': 
    appRank_p = appRank[:]
  else:
    appRank_a = appRank[:] 
  
  if kind == 'playstore': 
    p_appRank_free_2019,p_appRank_pay_2019,p_appRank_sales_2019 = separateAppRank(appRank)
    print('sep is done')
    p_appGenre_free_2019,p_appGenre_pay_2019,p_appGenre_sales_2019  = separateAppRank(appGenre)
    print('sep2 is done') 
  else:
    a_appRank_free_2019,a_appRank_pay_2019,a_appRank_sales_2019 = separateAppRank(appRank)
    print('sep is done')
    a_appGenre_free_2019,a_appGenre_pay_2019,a_appGenre_sales_2019  = separateAppRank(appGenre)
    print('sep2 is done') 

KeyboardInterrupt: ignored

In [ ]:
#2020
appRank = []
date_2020 = ['2020-01-31','2020-02-29','2020-03-31','2020-04-30','2020-05-31','2020-06-30']
date_size = len(date_2020)
kinds = ['playstore','appstore']

for kind in kinds:
  appRank = []
  appGenre = []
  for i in range(date_size):
    url = makeUrl(date_2020,i,kind)
    tmp = getAppRank(url,2020)
    appRank.append(tmp)
    genreURL = getGenreUrl(url)
    genreList = list()
    for j in range(len(genreURL)):
      genreList.append(get_genre('https://www.mobileindex.com/'+genreURL[j]['href'],tmp[j]))
    appGenre.append(genreList)
    print(kind,' date of ',date_2020[i],' is done')
  
  if kind == 'playstore': 
    p_appRank_free_2020,p_appRank_pay_2020,p_appRank_sales_2020 = separateAppRank(appRank)
    print('sep is done')
    p_appGenre_free_2020,p_appGenre_pay_2020,p_appGenre_sales_2020  = separateAppRank(appGenre)
    print('sep2 is done') 
  else:
    a_appRank_free_2020,a_appRank_pay_2020,a_appRank_sales_2020 = separateAppRank(appRank)
    print('sep is done')
    a_appGenre_free_2020,a_appGenre_pay_2020,a_appGenre_sales_2020  = separateAppRank(appGenre)
    print('sep2 is done') 

In [ ]:
p_appRank_free = list()
for i in range(len(p_appRank_free_2019)):
  p_appRank_free.append(p_appRank_free_2019[i])
  p_appRank_free.append(p_appGenre_free_2019[i])
for i in range(len(p_appRank_free_2020)):
  p_appRank_free.append(p_appRank_free_2020[i])
  p_appRank_free.append(p_appGenre_free_2020[i])

a_appRank_free = list()
for i in range(len(a_appRank_free_2019)):
  a_appRank_free.append(a_appRank_free_2019[i])
  a_appRank_free.append(a_appGenre_free_2019[i])
for i in range(len(a_appRank_free_2020)):
  a_appRank_free.append(a_appRank_free_2020[i])
  a_appRank_free.append(a_appGenre_free_2020[i])

col = []
for i in range(12):
  col.append('19_'+str(i+1))
  col.append('19_'+str(i+1)+'_gen')
for i in range(6):
  col.append('20_'+str(i+1))
  col.append('20_'+str(i+1)+'_gen')

appRank_free_playstore_df = pd.DataFrame(p_appRank_free)
appRank_free_playstore_df = appRank_free_playstore_df.T
appRank_free_playstore_df.columns = col
appRank_free_playstore_df.to_csv('/content/gdrive/My Drive/dacon/appRank_free_playstore_2019.csv', encoding='utf-8-sig')

appRank_free_appstore_df = pd.DataFrame(a_appRank_free)
appRank_free_appstore_df = appRank_free_appstore_df.T
appRank_free_appstore_df.columns = col
appRank_free_appstore_df.to_csv('/content/gdrive/My Drive/dacon/appRank_free_appstore_2020.csv', encoding='utf-8-sig')


In [ ]:
p_appRank_pay = list()
for i in range(len(p_appRank_pay_2019)):
  p_appRank_pay.append(p_appRank_pay_2019[i])
  p_appRank_pay.append(p_appGenre_pay_2019[i])
for i in range(len(p_appRank_pay_2020)):
  p_appRank_pay.append(p_appRank_pay_2020[i])
  p_appRank_pay.append(p_appGenre_pay_2020[i])

a_appRank_pay = list()
for i in range(len(a_appRank_pay_2019)):
  a_appRank_pay.append(a_appRank_pay_2019[i])
  a_appRank_pay.append(a_appGenre_pay_2019[i])
for i in range(len(a_appRank_pay_2020)):
  a_appRank_pay.append(a_appRank_pay_2020[i])
  a_appRank_pay.append(a_appGenre_pay_2020[i])

appRank_pay_playstore_df = pd.DataFrame(p_appRank_pay)
appRank_pay_playstore_df = appRank_pay_playstore_df.T
appRank_pay_playstore_df.columns = col
appRank_pay_playstore_df.to_csv('/content/gdrive/My Drive/dacon/appRank_pay_playstore_2019.csv', encoding='utf-8-sig')

appRank_pay_appstore_df = pd.DataFrame(a_appRank_pay)
appRank_pay_appstore_df = appRank_pay_appstore_df.T
appRank_pay_appstore_df.columns = col
appRank_pay_appstore_df.to_csv('/content/gdrive/My Drive/dacon/appRank_pay_appstore_2020.csv', encoding='utf-8-sig')

In [ ]:
p_appRank_sales = list()
for i in range(len(p_appRank_sales_2019)):
  p_appRank_sales.append(p_appRank_sales_2019[i])
  p_appRank_sales.append(p_appGenre_sales_2019[i])
for i in range(len(p_appRank_sales_2020)):
  p_appRank_sales.append(p_appRank_sales_2020[i])
  p_appRank_sales.append(p_appGenre_sales_2020[i])

a_appRank_sales = list()
for i in range(len(a_appRank_sales_2019)):
  a_appRank_sales.append(a_appRank_sales_2019[i])
  a_appRank_sales.append(a_appGenre_sales_2019[i])
for i in range(len(a_appRank_sales_2020)):
  a_appRank_sales.append(a_appRank_sales_2020[i])
  a_appRank_sales.append(a_appGenre_sales_2020[i])

p_sales_rank_df = pd.DataFrame(p_appRank_sales)
p_sales_rank_df = p_sales_rank_df.T
p_sales_rank_df.columns = col[12:]
p_sales_rank_df.to_csv('/content/gdrive/My Drive/app_sales_rank_playstore_2019.csv', encoding='utf-8-sig')

a_sales_rank_df = pd.DataFrame(a_appRank_sales)
a_sales_rank_df = a_sales_rank_df.T
a_sales_rank_df.columns = col
a_sales_rank_df.to_csv('/content/gdrive/My Drive/dacon/app_sales_rank_appstore_2020.csv', encoding='utf-8-sig')

In [ ]:
appRank_free_playstore_19_df = pd.read_csv('/content/gdrive/My Drive/dacon/app/appRank_free_playstore_2019.csv')
appRank_free_appstore_19_df = pd.read_csv('/content/gdrive/My Drive/dacon/app/appRank_free_appstore_2019.csv')
appRank_pay_playstore_19_df = pd.read_csv('/content/gdrive/My Drive/dacon/app/appRank_pay_playstore_2019.csv')
appRank_pay_appstore_19_df = pd.read_csv('/content/gdrive/My Drive/dacon/app/appRank_pay_appstore_2019.csv')
appRank_free_playstore_20_df = pd.read_csv('/content/gdrive/My Drive/dacon/app/appRank_free_playstore_2020.csv')
appRank_free_appstore_20_df = pd.read_csv('/content/gdrive/My Drive/dacon/app/appRank_free_appstore_2020.csv')
appRank_pay_playstore_20_df = pd.read_csv('/content/gdrive/My Drive/dacon/app/appRank_pay_playstore_2020.csv')
appRank_pay_appstore_20_df = pd.read_csv('/content/gdrive/My Drive/dacon/app/appRank_pay_appstore_2020.csv')
appRank_sales_playstore_20_df = pd.read_csv('/content/gdrive/My Drive/dacon/app/app_sales_rank_playstore_2020.csv')
appRank_sales_appstore_20_df = pd.read_csv('/content/gdrive/My Drive/dacon/app/app_sales_rank_appstore_2020.csv')
appRank_sales_playstore_19_df = pd.read_csv('/content/gdrive/My Drive/dacon/app/app_sales_rank_playstore_2019.csv')
appRank_sales_appstore_19_df = pd.read_csv('/content/gdrive/My Drive/dacon/app/app_sales_rank_appstore_2019.csv')

#### 플레이스토어 무료

In [ ]:
appRank_free_playstore_19_df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [ ]:
tmp = [i for i in list(appRank_free_playstore_19_df) if 'gen' in i]
free_p_gen = list(set(np.array(appRank_free_playstore_19_df[tmp]).reshape(len(tmp)*50,)))
free_p_gen_dic_19 = []
for i in tmp:
  free_p_gen_dic_tmp = {key: 0 for key in dict.fromkeys(free_p_gen).keys()}
  score = 50
  for j in appRank_free_playstore_19_df.dropna()[i]:
    free_p_gen_dic_tmp[j] = free_p_gen_dic_tmp[j]+score
    score -= 1
  free_p_gen_dic_19.append(free_p_gen_dic_tmp)

col = []
for i in range(12):
  col.append('19_'+str(i+1))

free_playstore_19 = pd.DataFrame(free_p_gen_dic_19)
free_playstore_19 = free_playstore_19.T
free_playstore_19.columns = col
free_playstore_19 = free_playstore_19.drop([free_playstore_19.index[0]])
free_playstore_19.head()

,19_1,19_2,19_3,19_4,19_5,19_6,19_7,19_8,19_9,19_10,19_11,19_12
라이프스타일,0,0,22,2,51,49,35,59,18,0,20,0
여행 및 지역정보,59,88,69,120,90,104,117,92,108,108,147,158
자동차,0,0,0,0,0,11,0,0,0,0,0,0
예술/디자인,0,0,0,7,0,0,0,0,0,0,0,0
교육,4,48,3,11,0,0,0,0,0,0,0,0


In [ ]:
free_playstore_19.dtypes

19_1     int64
19_2     int64
19_3     int64
19_4     int64
19_5     int64
19_6     int64
19_7     int64
19_8     int64
19_9     int64
19_10    int64
19_11    int64
19_12    int64
dtype: object

In [ ]:
free_p_standard_19 = free_playstore_19[['19_1','19_2','19_3','19_4','19_5','19_6','19_7','19_8','19_9']]
free_p_standard_19 = free_p_standard_19.reset_index()
free_p_standard_19 = free_p_standard_19.set_index('index')

free_p_standard_19['average'] = 0.000
print(len(free_p_standard_19))
for i in range(24):
  total = 0
  for j in range(0,9):
    total += free_p_standard_19.iloc[i][j]
  free_p_standard_19['average'][i] = total/9

free_p_standard_19 = free_p_standard_19[['average']]
free_p_standard_19 = free_p_standard_19.reset_index()
genre = list(free_p_standard_19['index'])
free_p_standard_19 = free_p_standard_19.set_index('index')
free_p_standard_19.head()

24


,average
index,
라이프스타일,26.222222
여행 및 지역정보,94.111111
자동차,1.222222
예술/디자인,0.777778
교육,7.333333


20년의 무료, 플레이스토어 앱 데이터를 불러와 19년도와 같은 가공 과정을 거칩니다. 

In [ ]:
free_playstore_19 = free_playstore_19[['19_10','19_11','19_12']]

col = []
for i in range(10,13):
  col.append(str(i)+'월')

free_playstore_19 = free_playstore_19.reset_index()

for i in range(len(col)):
  free_playstore_19[col[i]] = 0.000

for i in range(len(free_playstore_19)):
  base = free_playstore_19['index'][i]
  if base in genre:
    idx = genre.index(base)
    base = free_p_standard_19.iloc[idx]['average']
  else:
    base = 1.0
  for j in range(2, 5):
    if base == 0:
      value = 0.0
    else:
      value = (float(free_playstore_19.iloc[i][j]) - base) / base
    free_playstore_19[col[j-2]][i] = value

free_playstore_19 = free_playstore_19.set_index('index')
free_playstore_19 = free_playstore_19[['10월','11월','12월']]
free_playstore_19 = free_playstore_19.reset_index()
free_playstore_19.columns = ['genre', '10월','11월','12월']
month = list()
for i in range(10,13):
  month.append(list(free_playstore_19[str(i)+'월']))
genre_list = list(free_playstore_19['genre'])

free_playstore_19_graph = pd.DataFrame()
for i in range(3):
  free_playstore_19_fig = pd.DataFrame(genre_list,columns = ['genre'])
  free_playstore_19_fig['value'] = month[i]
  free_playstore_19_fig['month'] = str(i+10)+'월'
  free_playstore_19_fig = free_playstore_19_fig[['genre','month','value']]
  free_playstore_19_graph = pd.concat([free_playstore_19_graph,free_playstore_19_fig], axis = 0)

free_playstore_19_graph

,genre,month,value
0,라이프스타일,10월,-0.237288
1,여행 및 지역정보,10월,0.561983
2,자동차,10월,-1.000000
3,예술/디자인,10월,-1.000000
4,교육,10월,-1.000000
...,...,...,...
19,건강/운동,12월,3.080000
20,동영상 플레이어/편집기,12월,-1.025210
21,금융,12월,-0.998537
22,부동산/홈 인테리어,12월,-1.024917


In [ ]:
appRank_free_playstore_20_df.drop(['Unnamed: 0'], axis = 1, inplace=True)

tmp = [i for i in list(appRank_free_playstore_20_df) if 'gen' in i]
free_p_gen = list(set(np.array(appRank_free_playstore_20_df[tmp]).reshape(len(tmp)*50,)))
free_p_gen_dic_20 = []
for i in tmp:
  free_p_gen_dic_tmp = {key: 0 for key in dict.fromkeys(free_p_gen).keys()}
  score = 50
  for j in appRank_free_playstore_20_df.dropna()[i]:
    free_p_gen_dic_tmp[j] = free_p_gen_dic_tmp[j]+score
    score -= 1
  free_p_gen_dic_20.append(free_p_gen_dic_tmp)

col = []
for i in range(6):
  col.append('20_'+str(i+1))

free_playstore_20 = pd.DataFrame(free_p_gen_dic_20)
free_playstore_20 = free_playstore_20.T
free_playstore_20.columns = col
free_playstore_20 = free_playstore_20.drop([free_playstore_20.index[0]])

col = []
for i in range(1,7):
  col.append(str(i)+'월')

free_playstore_20 = free_playstore_20.reset_index()

In [ ]:
for i in range(len(col)):
  free_playstore_20[col[i]] = 0.000
# 
for i in range(len(free_playstore_20)):
  base = free_playstore_20['index'][i]
  if base in genre:
    idx = genre.index(base)
    base = free_p_standard_19.iloc[idx]['average']
    if base == 0.0:
      base = 1.0
  else:
    base = 1.0
  for j in range(1, 7):
    if base == 0:
      value = 0.0
    else:
      value = (float(free_playstore_20.iloc[i][j]) - base) / base
    free_playstore_20[col[j-1]][i] = value

In [ ]:
free_playstore_20 = free_playstore_20.set_index('index')
free_playstore_20 = free_playstore_20[['1월','2월','3월','4월','5월','6월']]
free_playstore_20 = free_playstore_20.reset_index()
free_playstore_20.columns = ['genre', '1월','2월','3월','4월','5월','6월']
month = list()
for i in range(1,7):
  month.append(list(free_playstore_20[str(i)+'월']))
genre_list = list(free_playstore_20['genre'])

In [ ]:
free_playstore_20_graph = pd.DataFrame()
for i in range(6):
  free_playstore_20_fig = pd.DataFrame(genre_list,columns = ['genre'])
  free_playstore_20_fig['value'] = month[i]
  free_playstore_20_fig['month'] = str(i+1)+'월'
  free_playstore_20_fig = free_playstore_20_fig[['genre','month','value']]
  free_playstore_20_graph = pd.concat([free_playstore_20_graph,free_playstore_20_fig], axis = 0)

free_playstore_20_graph

,genre,month,value
0,여행 및 지역정보,1월,-0.033058
1,자동차,1월,-1.000000
2,교육,1월,-1.000000
3,비즈니스,1월,-1.000000
4,식음료,1월,-0.046444
...,...,...,...
18,동영상 플레이어/편집기,6월,1.495798
19,금융,6월,-0.581034
20,부동산/홈 인테리어,6월,1.214660
21,의료,6월,-1.000000


In [ ]:
free_playstore_19_20_graph = pd.concat([free_playstore_19_graph, free_playstore_20_graph], axis = 0)
free_playstore_19_20_graph

,genre,month,value
0,라이프스타일,10월,-0.237288
1,여행 및 지역정보,10월,0.561983
2,자동차,10월,-1.000000
3,예술/디자인,10월,-1.000000
4,교육,10월,-1.000000
...,...,...,...
18,동영상 플레이어/편집기,6월,1.495798
19,금융,6월,-0.581034
20,부동산/홈 인테리어,6월,1.214660
21,의료,6월,-1.000000


In [ ]:
df = free_playstore_19_20_graph

fig = px.bar(df, x="genre", y="value", color="genre",
  animation_frame="month", animation_group="genre", range_y=[min(free_playstore_19_20_graph['value'])-5, max(free_playstore_19_20_graph['value'])+5])
fig.show()

#### 플레이스토어 유료

In [ ]:
appRank_pay_playstore_19_df.drop(['Unnamed: 0'], axis = 1, inplace= True)

tmp = [i for i in list(appRank_pay_playstore_19_df) if 'gen' in i]
pay_p_gen = list(set(np.array(appRank_pay_playstore_19_df[tmp]).reshape(len(tmp)*50,)))
pay_p_gen_dic_19 = []
for i in tmp:
  pay_p_gen_dic_tmp = {key: 0 for key in dict.fromkeys(pay_p_gen).keys()}
  score = 50
  for j in appRank_pay_playstore_19_df.dropna()[i]:
    pay_p_gen_dic_tmp[j] = pay_p_gen_dic_tmp[j]+score
    score -= 1
  pay_p_gen_dic_19.append(pay_p_gen_dic_tmp)

col = []
for i in range(12):
  col.append('19_'+str(i+1))

pay_playstore_19 = pd.DataFrame(pay_p_gen_dic_19)
pay_playstore_19 = pay_playstore_19.T
pay_playstore_19.columns = col
pay_playstore_19 = pay_playstore_19.drop([pay_playstore_19.index[0]])

In [ ]:
pay_p_standard_19 = pay_playstore_19[['19_1','19_2','19_3','19_4','19_5','19_6','19_7','19_8','19_9']]
pay_p_standard_19 = pay_p_standard_19.reset_index()
pay_p_standard_19 = pay_p_standard_19.set_index('index')

pay_p_standard_19['average'] = 0.000
for i in range(len(pay_p_standard_19)):
  total = 0
  for j in range(0,9):
    total += pay_p_standard_19.iloc[i][j]
  pay_p_standard_19['average'][i] = total/9

pay_p_standard_19 = pay_p_standard_19[['average']]
pay_p_standard_19 = pay_p_standard_19.reset_index()
genre = list(pay_p_standard_19['index'])
pay_p_standard_19 = pay_p_standard_19.set_index('index')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
pay_playstore_19 = pay_playstore_19[['19_10','19_11','19_12']]

col = []
for i in range(10,13):
  col.append(str(i)+'월')

pay_playstore_19 = pay_playstore_19.reset_index()

for i in range(len(col)):
  pay_playstore_19[col[i]] = 0.000

for i in range(len(pay_playstore_19)):
  base = pay_playstore_19['index'][i]
  if base in genre:
    idx = genre.index(base)
    base = pay_p_standard_19.iloc[idx]['average']
  else:
    base = 1.0
  for j in range(2, 5):
    if base == 0:
      value = 0.0
    else:
      value = (float(pay_playstore_19.iloc[i][j]) - base) / base
    pay_playstore_19[col[j-2]][i] = value

pay_playstore_19 = pay_playstore_19.set_index('index')
pay_playstore_19 = pay_playstore_19[['10월','11월','12월']]
pay_playstore_19 = pay_playstore_19.reset_index()
pay_playstore_19.columns = ['genre', '10월','11월','12월']
month = list()
for i in range(10,13):
  month.append(list(pay_playstore_19[str(i)+'월']))
genre_list = list(pay_playstore_19['genre'])

pay_playstore_19_graph = pd.DataFrame()
for i in range(3):
  pay_playstore_19_fig = pd.DataFrame(genre_list,columns = ['genre'])
  pay_playstore_19_fig['value'] = month[i]
  pay_playstore_19_fig['month'] = str(i+10)+'월'
  pay_playstore_19_fig = pay_playstore_19_fig[['genre','month','value']]
  pay_playstore_19_graph = pd.concat([pay_playstore_19_graph,pay_playstore_19_fig], axis = 0)

pay_playstore_19_graph.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,genre,month,value
0,만화,10월,0.767857
1,도구,10월,-0.100592
2,생산성,10월,0.127517
3,음악/오디오,10월,0.242391
4,엔터테인먼트,10월,-1.000000


In [ ]:
appRank_pay_playstore_20_df.drop(['Unnamed: 0'], axis = 1, inplace=True)

tmp = [i for i in list(appRank_pay_playstore_20_df) if 'gen' in i]
pay_p_gen = list(set(np.array(appRank_pay_playstore_20_df[tmp]).reshape(len(tmp)*50,)))
pay_p_gen_dic_20 = []
for i in tmp:
  pay_p_gen_dic_tmp = {key: 0 for key in dict.fromkeys(pay_p_gen).keys()}
  score = 50
  for j in appRank_pay_playstore_20_df.dropna()[i]:
    pay_p_gen_dic_tmp[j] = pay_p_gen_dic_tmp[j]+score
    score -= 1
  pay_p_gen_dic_20.append(pay_p_gen_dic_tmp)

In [ ]:
col = []
for i in range(6):
  col.append('20_'+str(i+1))

pay_playstore_20 = pd.DataFrame(pay_p_gen_dic_20)
pay_playstore_20 = pay_playstore_20.T
pay_playstore_20.columns = col
pay_playstore_20 = pay_playstore_20.drop([pay_playstore_20.index[0]])

col = []
for i in range(1,7):
  col.append(str(i)+'월')

pay_playstore_20 = pay_playstore_20.reset_index()

for i in range(len(col)):
  pay_playstore_20[col[i]] = 0.000

for i in range(len(pay_playstore_20)):
  base = pay_playstore_20['index'][i]
  if base in genre:
    idx = genre.index(base)
    base = pay_p_standard_19.iloc[idx]['average']
    if base == 0.0:
      base = 1.0
  else:
    base = 1.0
  for j in range(1, 7):
    if base == 0:
      value = 0.0
    else:
      value = (float(pay_playstore_20.iloc[i][j]) - base) / base
    pay_playstore_20[col[j-1]][i] = value

pay_playstore_20 = pay_playstore_20.set_index('index')
pay_playstore_20 = pay_playstore_20[['1월','2월','3월','4월','5월','6월']]
pay_playstore_20 = pay_playstore_20.reset_index()
pay_playstore_20.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,index,1월,2월,3월,4월,5월,6월
0,만화,0.044643,0.285714,1.089286,0.607143,1.571429,1.089286
1,도구,-0.343195,-0.295858,-0.147929,-0.023669,-0.017751,0.201183
2,생산성,0.087248,0.550336,0.570470,0.194631,0.208054,0.134228
3,엔터테인먼트,2.642857,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
4,음악/오디오,-0.119565,0.095652,-0.188043,-0.246739,0.369565,0.193478


In [ ]:
month = list()
for i in range(1,7):
  month.append(list(pay_playstore_20[str(i)+'월']))

genre_list = list(pay_playstore_20['index'])

pay_playstore_20_graph = pd.DataFrame()
for i in range(6):
  pay_playstore_20_fig = pd.DataFrame(genre_list,columns = ['genre'])
  pay_playstore_20_fig['value'] = month[i]
  pay_playstore_20_fig['month'] = str(i+1)+'월'
  pay_playstore_20_fig = pay_playstore_20_fig[['genre','month','value']]
  pay_playstore_20_graph = pd.concat([pay_playstore_20_graph,pay_playstore_20_fig], axis = 0)

pay_playstore_20_graph.head()

,genre,month,value
0,만화,1월,0.044643
1,도구,1월,-0.343195
2,생산성,1월,0.087248
3,엔터테인먼트,1월,2.642857
4,음악/오디오,1월,-0.119565


In [ ]:
pay_playstore_19_20_graph = pd.concat([pay_playstore_19_graph, pay_playstore_20_graph], axis = 0)
pay_playstore_19_20_graph.head()

,genre,month,value
0,만화,10월,0.767857
1,도구,10월,-0.100592
2,생산성,10월,0.127517
3,음악/오디오,10월,0.242391
4,엔터테인먼트,10월,-1.000000


In [ ]:
df = pay_playstore_19_20_graph

fig = px.bar(df, x="genre", y="value", color="genre",
  animation_frame="month", animation_group="genre", range_y=[min(pay_playstore_19_20_graph['value'])-5, max(pay_playstore_19_20_graph['value'])+5])
fig.show()

#### 플레이스토어 매출

In [ ]:
appRank_sales_playstore_19_df.drop(['Unnamed: 0'],axis=1,inplace= True)

tmp = [i for i in list(appRank_sales_playstore_19_df) if 'gen' in i]
sales_p_gen = list(set(np.array(appRank_sales_playstore_19_df[tmp]).reshape(len(tmp)*50,)))
sales_p_gen_dic_19 = []
for i in tmp:
  sales_p_gen_dic_tmp = {key: 0 for key in dict.fromkeys(sales_p_gen).keys()}
  score = 50
  for j in appRank_sales_playstore_19_df.dropna()[i]:
    sales_p_gen_dic_tmp[j] = sales_p_gen_dic_tmp[j]+score
    score -= 1
  sales_p_gen_dic_19.append(sales_p_gen_dic_tmp)

In [ ]:
col = []
for i in range(6,12):
  col.append('19_'+str(i+1))

sales_playstore_19 = pd.DataFrame(sales_p_gen_dic_19)
sales_playstore_19 = sales_playstore_19.T
sales_playstore_19.columns = col
sales_playstore_19 = sales_playstore_19.drop([sales_playstore_19.index[0]])
sales_playstore_19 = sales_playstore_19.reset_index()
sales_playstore_19 = sales_playstore_19.set_index('index')

In [ ]:
sales_playstore_19['average'] = 0.000

for i in range(len(sales_playstore_19)):
  total = 0
  for j in range(0,6):
    total += sales_playstore_19.iloc[i][j]
  sales_playstore_19['average'][i] = total/9

sales_playstore_19 = sales_playstore_19[['average']]
sales_playstore_19 = sales_playstore_19.reset_index()
genre = list(sales_playstore_19['index'])
sales_playstore_19 = sales_playstore_19.set_index('index')
sales_playstore_19.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,average
index,
라이프스타일,49.666667
도구,4.555556
생산성,31.444444
음악/오디오,7.111111
도서/참고자료,25.666667


In [ ]:
appRank_sales_playstore_20_df.drop(['Unnamed: 0'], axis = 1, inplace=True)
tmp = [i for i in list(appRank_sales_playstore_20_df) if 'gen' in i]
sales_p_gen = list(set(np.array(appRank_sales_playstore_20_df[tmp]).reshape(len(tmp)*50,)))
sales_p_gen_dic_20 = []
for i in tmp:
  sales_p_gen_dic_tmp = {key: 0 for key in dict.fromkeys(sales_p_gen).keys()}
  score = 50
  for j in appRank_sales_playstore_20_df.dropna()[i]:
    sales_p_gen_dic_tmp[j] = sales_p_gen_dic_tmp[j]+score
    score -= 1
  sales_p_gen_dic_20.append(sales_p_gen_dic_tmp)

In [ ]:
col = []
for i in range(6):
  col.append('20_'+str(i+1))

sales_playstore_20 = pd.DataFrame(sales_p_gen_dic_20)
sales_playstore_20 = sales_playstore_20.T
sales_playstore_20.columns = col
sales_playstore_20 = sales_playstore_20.drop([sales_playstore_20.index[0]])
sales_playstore_20 = sales_playstore_20.reset_index()

In [ ]:
col = []
for i in range(1,7):
  col.append(str(i)+'월')

for i in range(len(col)):
  sales_playstore_20[col[i]] = 0.000

for i in range(len(sales_playstore_20)):
  base = sales_playstore_20['index'][i]
  if base in genre:
    idx = genre.index(base)
    base = sales_playstore_19.iloc[idx]['average']
    if base == 0.0:
      base = 1.0
  else:
    base = 1.0
  for j in range(1, 7):
    if base == 0:
      value = 0.0
    else:
      value = (float(sales_playstore_20.iloc[i][j]) - base) / base
    sales_playstore_20[col[j-1]][i] = value

sales_playstore_20 = sales_playstore_20.set_index('index')
sales_playstore_20 = sales_playstore_20[['1월','2월','3월','4월','5월','6월']]
sales_playstore_20 = sales_playstore_20.reset_index()
sales_playstore_20.columns = ['genre', '1월','2월','3월','4월','5월','6월']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
month = list()
for i in range(1,7):
  month.append(list(sales_playstore_20[str(i)+'월']))
genre_list = list(sales_playstore_20['genre'])

sales_playstore_20_graph = pd.DataFrame()
for i in range(6):
  sales_playstore_20_fig = pd.DataFrame(genre_list,columns = ['genre'])
  sales_playstore_20_fig['value'] = month[i]
  sales_playstore_20_fig['month'] = str(i+1)+'월'
  sales_playstore_20_fig = sales_playstore_20_fig[['genre','month','value']]
  sales_playstore_20_graph = pd.concat([sales_playstore_20_graph,sales_playstore_20_fig], axis = 0)

sales_playstore_20_graph.head()

,genre,month,value
0,만화,1월,-1.000000
1,생산성,1월,0.494700
2,음악/오디오,1월,-0.156250
3,도서/참고자료,1월,0.363636
4,커뮤니케이션,1월,0.616279


In [ ]:
df = sales_playstore_20_graph

fig = px.bar(df, x="genre", y="value", color="genre",
  animation_frame="month", animation_group="genre", range_y=[min(sales_playstore_20_graph['value'])-5, max(sales_playstore_20_graph['value'])+5])
fig.show()

#### 앱스토어 무료

In [ ]:
appRank_free_appstore_19_df.drop(['Unnamed: 0'],axis=1,inplace=True)

tmp = [i for i in list(appRank_free_appstore_19_df) if 'gen' in i]
free_a_gen = list(set(np.array(appRank_free_appstore_19_df[tmp]).reshape(len(tmp)*100,)))
free_a_gen_dic_19 = []
for i in tmp:
  free_a_gen_dic_tmp = {key: 0 for key in dict.fromkeys(free_a_gen).keys()}
  score = 100
  for j in appRank_free_appstore_19_df.dropna()[i]:
    free_a_gen_dic_tmp[j] = free_a_gen_dic_tmp[j]+score
    score -= 1
  free_a_gen_dic_19.append(free_a_gen_dic_tmp)

In [ ]:
print(free_a_gen_dic_19)

[{nan: 0, '라이프스타일': 56, '자동차 경주': 46, 'Social Networking': 0, 'Simulation': 0, '롤플레잉\n  (일 경과)': 0, 'Role Playing': 0, '여행 및 지역정보': 425, 'Entertainment': 66, 'Finance': 0, 'Trivia': 0, 'Navigation': 0, '시뮬레이션': 56, 'Board': 0, 'Adventure': 0, 'Games': 0, '교육': 0, 'Family': 48, '식음료': 147, '비즈니스': 29, 'Puzzle': 0, '지도/내비게이션': 211, '캐주얼 게임': 0, '음악': 100, '아케이드': 178, '보드': 0, '만화': 40, '비즈니스\n  (일 경과)': 55, '카드': 0, '생산성': 54, '음악/오디오': 67, '시뮬레이션\n  (일 경과)': 0, '도서/참고자료': 109, '소셜': 256, '전략': 0, '뷰티': 24, '스포츠': 0, 'Productivity': 0, 'Card': 0, '날씨': 0, '쇼핑': 524, 'Action': 58, 'Lifestyle': 0, '사진': 119, '퀴즈': 0, '액션': 161, '롤플레잉': 448, '도구': 163, 'Food & Drink': 0, 'Photo & Video': 99, '카지노': 0, 'Utilities': 0, '커뮤니케이션': 139, '건강/운동': 20, '퍼즐': 0, '어드벤처': 80, '동영상 플레이어/편집기': 203, '라이프스타일\n  (일 경과)': 0, '금융': 381, '부동산/홈 인테리어': 216, '퍼즐\n  (일 경과)': 0, '엔터테인먼트': 319, '\n  (일 경과)\n-\n\n': 0}, {nan: 0, '라이프스타일': 18, '자동차 경주': 51, 'Social Networking': 0, 'Simulation': 0, '롤플레잉\n  (일 경과)':

In [ ]:
col = []
for i in range(12):
  col.append('19_'+str(i+1))

free_appstore_19 = pd.DataFrame(free_a_gen_dic_19)
free_appstore_19 = free_appstore_19.T
free_appstore_19.columns = col
free_appstore_19 = free_appstore_19.drop([free_appstore_19.index[0]])

free_a_standard_19 = free_appstore_19[['19_1','19_2','19_3','19_4','19_5','19_6','19_7','19_8','19_9']]
free_a_standard_19 = free_a_standard_19.reset_index()
free_a_standard_19 = free_a_standard_19.set_index('index')
free_a_standard_19.head()

,19_1,19_2,19_3,19_4,19_5,19_6,19_7,19_8,19_9
index,,,,,,,,,
라이프스타일,56,18,100,161,367,120,0,71,65
자동차 경주,46,51,0,0,0,0,0,0,0
Social Networking,0,0,0,0,0,0,0,0,0
Simulation,0,0,72,163,0,0,0,0,0
롤플레잉\n (일 경과),0,0,34,0,0,0,0,0,0


In [ ]:
free_a_standard_19['average'] = 0.000
for i in range(len(free_a_standard_19)):
  total = 0
  for j in range(0,9):
    total += free_a_standard_19.iloc[i][j]
  free_a_standard_19['average'][i] = total/9

free_a_standard_19 = free_a_standard_19[['average']]
free_a_standard_19 = free_a_standard_19.reset_index()
genre = list(free_a_standard_19['index'])
free_a_standard_19 = free_a_standard_19.set_index('index')
free_a_standard_19

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,average
index,
라이프스타일,106.444444
자동차 경주,10.777778
Social Networking,0.000000
Simulation,26.111111
롤플레잉\n (일 경과),3.777778
...,...
금융,432.333333
부동산/홈 인테리어,99.222222
퍼즐\n (일 경과),7.444444


In [ ]:
free_appstore_19 = free_appstore_19[['19_10','19_11','19_12']]

col = []
for i in range(10,13):
  col.append(str(i)+'월')

free_appstore_19 = free_appstore_19.reset_index()

for i in range(len(col)):
  free_appstore_19[col[i]] = 0.000

for i in range(len(free_appstore_19)):
  base = free_appstore_19['index'][i]
  if base in genre:
    idx = genre.index(base)
    base = free_a_standard_19.iloc[idx]['average']
  else:
    base = 1.0
  for j in range(2, 5):
    if base == 0:
      value = 0.0
    else:
      value = (float(free_appstore_19.iloc[i][j]) - base) / base
    free_appstore_19[col[j-2]][i] = value

free_appstore_19 = free_appstore_19.set_index('index')
free_appstore_19 = free_appstore_19[['10월','11월','12월']]
free_appstore_19 = free_appstore_19.reset_index()
free_appstore_19.columns = ['genre', '10월','11월','12월']
month = list()
for i in range(10,13):
  month.append(list(free_appstore_19[str(i)+'월']))
genre_list = list(free_appstore_19['genre'])

free_appstore_19_graph = pd.DataFrame()
for i in range(3):
  free_appstore_19_fig = pd.DataFrame(genre_list,columns = ['genre'])
  free_appstore_19_fig['value'] = month[i]
  free_appstore_19_fig['month'] = str(i+10)+'월'
  free_appstore_19_fig = free_appstore_19_fig[['genre','month','value']]
  free_appstore_19_graph = pd.concat([free_appstore_19_graph,free_appstore_19_fig], axis = 0)

free_appstore_19_graph.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,genre,month,value
0,라이프스타일,10월,0.136743
1,자동차 경주,10월,-1.000000
2,Social Networking,10월,0.000000
3,Simulation,10월,-1.000000
4,롤플레잉\n (일 경과),10월,-1.000000


In [ ]:
appRank_free_appstore_20_df.drop(['Unnamed: 0'], axis = 1, inplace=True)

tmp = [i for i in list(appRank_free_appstore_20_df) if 'gen' in i]
free_a_gen = list(set(np.array(appRank_free_appstore_20_df[tmp]).reshape(len(tmp)*100,)))
free_a_gen_dic_20 = []
for i in tmp:
  free_a_gen_dic_tmp = {key: 0 for key in dict.fromkeys(free_a_gen).keys()}
  score = 100
  for j in appRank_free_appstore_20_df.dropna()[i]:
    free_a_gen_dic_tmp[j] = free_a_gen_dic_tmp[j]+score
    score -= 1
  free_a_gen_dic_20.append(free_a_gen_dic_tmp)

In [ ]:
col = []
for i in range(6):
  col.append('20_'+str(i+1))

free_appstore_20 = pd.DataFrame(free_a_gen_dic_20)
free_appstore_20 = free_appstore_20.T
free_appstore_20.columns = col
free_appstore_20 = free_appstore_20.drop([free_appstore_20.index[0]])
free_appstore_20 = free_appstore_20.reset_index()

In [ ]:
col = []
for i in range(1,7):
  col.append(str(i)+'월')

for i in range(len(col)):
  free_appstore_20[col[i]] = 0.000

for i in range(len(free_appstore_20)):
  base = free_appstore_20['index'][i]
  if base in genre:
    idx = genre.index(base)
    base = free_a_standard_19.iloc[idx]['average']
    if base == 0.0:
      base = 1.0
  else:
    base = 1.0
  for j in range(1, 7):
    if base == 0:
      value = 0.0
    else:
      value = (float(free_appstore_20.iloc[i][j]) - base) / base
    free_appstore_20[col[j-1]][i] = value

free_appstore_20 = free_appstore_20.set_index('index')
free_appstore_20 = free_appstore_20[['1월','2월','3월','4월','5월','6월']]
free_appstore_20 = free_appstore_20.reset_index()
free_appstore_20.columns = ['genre', '1월','2월','3월','4월','5월','6월']
free_appstore_20.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,genre,1월,2월,3월,4월,5월,6월
0,라이프스타일,0.888309,-1.0,-0.511482,-0.727557,0.822547,0.371608
1,자동차 경주,-1.000000,-1.0,-1.000000,-1.000000,8.278351,8.278351
2,여행 및 지역정보,0.101353,-1.0,-0.771955,-0.512813,-0.406565,-0.248488
3,캐주얼 게임\n (일 경과),-1.000000,-1.0,-1.000000,61.000000,-1.000000,-1.000000
4,Navigation,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


In [ ]:
month = list()
for i in range(1,7):
  month.append(list(free_appstore_20[str(i)+'월']))

genre_list = list(free_appstore_20['genre'])

free_appstore_20_graph = pd.DataFrame()
for i in range(6):
  free_appstore_20_fig = pd.DataFrame(genre_list,columns = ['genre'])
  free_appstore_20_fig['value'] = month[i]
  free_appstore_20_fig['month'] = str(i+1)+'월'
  free_appstore_20_fig = free_appstore_20_fig[['genre','month','value']]
  free_appstore_20_graph = pd.concat([free_appstore_20_graph,free_appstore_20_fig], axis = 0)

free_appstore_20_graph.head()

,genre,month,value
0,라이프스타일,1월,0.888309
1,자동차 경주,1월,-1.000000
2,여행 및 지역정보,1월,0.101353
3,캐주얼 게임\n (일 경과),1월,-1.000000
4,Navigation,1월,0.000000


In [ ]:
free_appstore_19_20_graph = pd.concat([free_appstore_19_graph, free_appstore_20_graph], axis = 0)
df = free_appstore_19_20_graph

fig = px.bar(df, x="genre", y="value", color="genre",
  animation_frame="month", animation_group="genre", range_y=[min(free_appstore_19_20_graph['value'])-5, max(free_appstore_19_20_graph['value'])+5])
fig.show()

#### 앱스토어 유료

In [ ]:
appRank_pay_appstore_19_df.drop(['Unnamed: 0'], axis = 1, inplace= True)

tmp = [i for i in list(appRank_pay_appstore_19_df) if 'gen' in i]
pay_a_gen = list(set(np.array(appRank_pay_appstore_19_df[tmp]).reshape(len(tmp)*100,)))
pay_a_gen_dic_19 = []
for i in tmp:
  pay_a_gen_dic_tmp = {key: 0 for key in dict.fromkeys(pay_a_gen).keys()}
  score = 100
  for j in appRank_pay_appstore_19_df.dropna()[i]:
    pay_a_gen_dic_tmp[j] = pay_a_gen_dic_tmp[j]+score
    score -= 1
  pay_a_gen_dic_19.append(pay_a_gen_dic_tmp)

In [ ]:
col = []
for i in range(12):
  col.append('19_'+str(i+1))

pay_appstore_19 = pd.DataFrame(pay_a_gen_dic_19)
pay_appstore_19 = pay_appstore_19.T
pay_appstore_19.columns = col
pay_appstore_19 = pay_appstore_19.drop([pay_appstore_19.index[0]])

pay_a_standard_19 = pay_appstore_19[['19_1','19_2','19_3','19_4','19_5','19_6','19_7','19_8','19_9']]
pay_a_standard_19 = pay_a_standard_19.reset_index()
pay_a_standard_19 = pay_a_standard_19.set_index('index')

In [ ]:
pay_a_standard_19['average'] = 0.000
for i in range(len(pay_a_standard_19)):
  total = 0
  for j in range(0,9):
    total += pay_a_standard_19.iloc[i][j]
  pay_a_standard_19['average'][i] = total/9

pay_a_standard_19 = pay_a_standard_19[['average']]
pay_a_standard_19 = pay_a_standard_19.reset_index()
genre = list(pay_a_standard_19['index'])
pay_a_standard_19 = pay_a_standard_19.set_index('index')
pay_a_standard_19.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,average
index,
라이프스타일,74.000000
자동차 경주,44.666667
의료\n (일 경과),0.000000
Social Networking,47.444444
Simulation,19.777778


In [ ]:
pay_appstore_19 = pay_appstore_19[['19_10','19_11','19_12']]

col = []
for i in range(10,13):
  col.append(str(i)+'월')

pay_appstore_19 = pay_appstore_19.reset_index()

for i in range(len(col)):
  pay_appstore_19[col[i]] = 0.000

for i in range(len(pay_appstore_19)):
  base = pay_appstore_19['index'][i]
  if base in genre:
    idx = genre.index(base)
    base = pay_a_standard_19.iloc[idx]['average']
  else:
    base = 1.0
  for j in range(2, 5):
    if base == 0:
      value = 0.0
    else:
      value = (float(pay_appstore_19.iloc[i][j]) - base) / base
    pay_appstore_19[col[j-2]][i] = value

pay_appstore_19 = pay_appstore_19.set_index('index')
pay_appstore_19 = pay_appstore_19[['10월','11월','12월']]
pay_appstore_19 = pay_appstore_19.reset_index()
pay_appstore_19.columns = ['genre', '10월','11월','12월']
month = list()
for i in range(10,13):
  month.append(list(pay_appstore_19[str(i)+'월']))
genre_list = list(pay_appstore_19['genre'])

pay_appstore_19_graph = pd.DataFrame()
for i in range(3):
  pay_appstore_19_fig = pd.DataFrame(genre_list,columns = ['genre'])
  pay_appstore_19_fig['value'] = month[i]
  pay_appstore_19_fig['month'] = str(i+10)+'월'
  pay_appstore_19_fig = pay_appstore_19_fig[['genre','month','value']]
  pay_appstore_19_graph = pd.concat([pay_appstore_19_graph,pay_appstore_19_fig], axis = 0)

pay_appstore_19_graph.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,genre,month,value
0,라이프스타일,10월,-1.000000
1,자동차 경주,10월,1.014925
2,의료\n (일 경과),10월,0.000000
3,Social Networking,10월,-0.409836
4,Simulation,10월,-1.000000


In [ ]:
appRank_pay_appstore_20_df.drop(['Unnamed: 0'], axis = 1, inplace=True)

tmp = [i for i in list(appRank_pay_appstore_20_df) if 'gen' in i]
pay_a_gen = list(set(np.array(appRank_pay_appstore_20_df[tmp]).reshape(len(tmp)*100,)))
pay_a_gen_dic_20 = []
for i in tmp:
  pay_a_gen_dic_tmp = {key: 0 for key in dict.fromkeys(pay_a_gen).keys()}
  score = 100
  for j in appRank_pay_appstore_20_df.dropna()[i]:
    pay_a_gen_dic_tmp[j] = pay_a_gen_dic_tmp[j]+score
    score -= 1
  pay_a_gen_dic_20.append(pay_a_gen_dic_tmp)

In [ ]:
col = []
for i in range(6):
  col.append('20_'+str(i+1))

pay_appstore_20 = pd.DataFrame(pay_a_gen_dic_20)
pay_appstore_20 = pay_appstore_20.T
pay_appstore_20.columns = col
pay_appstore_20 = pay_appstore_20.drop([pay_appstore_20.index[0]])
pay_appstore_20.head()

col = []
for i in range(1,7):
  col.append(str(i)+'월')

pay_appstore_20 = pay_appstore_20.reset_index()

In [ ]:
for i in range(len(col)):
  pay_appstore_20[col[i]] = 0.000

for i in range(len(pay_appstore_20)):
  base = pay_appstore_20['index'][i]
  if base in genre:
    idx = genre.index(base)
    base = pay_a_standard_19.iloc[idx]['average']
    if base == 0.0:
      base = 1.0
  else:
    base = 1.0
  for j in range(1, 7):
    if base == 0:
      value = 0.0
    else:
      value = (float(pay_appstore_20.iloc[i][j]) - base) / base
    pay_appstore_20[col[j-1]][i] = value

pay_appstore_20 = pay_appstore_20.set_index('index')
pay_appstore_20 = pay_appstore_20[['1월','2월','3월','4월','5월','6월']]
pay_appstore_20 = pay_appstore_20.reset_index()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
month = list()
for i in range(1,7):
  month.append(list(pay_appstore_20[str(i)+'월']))

genre_list = list(pay_appstore_20['index'])

pay_appstore_20_graph = pd.DataFrame()
for i in range(6):
  pay_appstore_20_fig = pd.DataFrame(genre_list,columns = ['genre'])
  pay_appstore_20_fig['value'] = month[i]
  pay_appstore_20_fig['month'] = str(i+1)+'월'
  pay_appstore_20_fig = pay_appstore_20_fig[['genre','month','value']]
  pay_appstore_20_graph = pd.concat([pay_appstore_20_graph,pay_appstore_20_fig], axis = 0)

pay_appstore_20_graph.head()

,genre,month,value
0,라이프스타일,1월,-1.000000
1,자동차 경주,1월,0.029851
2,Social Networking,1월,-1.000000
3,Role Playing,1월,-1.000000
4,Entertainment,1월,0.288325


In [ ]:
pay_appstore_19_20_graph = pd.concat([pay_appstore_19_graph,pay_appstore_20_graph], axis = 0)
df =pay_appstore_19_20_graph

fig = px.bar(df, x="genre", y="value", color="genre",
  animation_frame="month", animation_group="genre", range_y=[min(pay_appstore_19_20_graph['value'])-5, max(pay_appstore_19_20_graph['value'])+5])
fig.show()

#### 앱스토어 매출

In [ ]:
appRank_sales_appstore_19_df.drop(['Unnamed: 0'],axis=1,inplace= True)

tmp = [i for i in list(appRank_sales_appstore_19_df) if 'gen' in i]
sales_a_gen = list(set(np.array(appRank_sales_appstore_19_df[tmp]).reshape(len(tmp)*100,)))
sales_a_gen_dic_19 = []
for i in tmp:
  sales_a_gen_dic_tmp = {key: 0 for key in dict.fromkeys(sales_a_gen).keys()}
  score = 100
  for j in appRank_sales_appstore_19_df.dropna()[i]:
    sales_a_gen_dic_tmp[j] = sales_a_gen_dic_tmp[j]+score
    score -= 1
  sales_a_gen_dic_19.append(sales_a_gen_dic_tmp)

In [ ]:
col = []
for i in range(12):
  col.append('19_'+str(i+1))

sales_appstore_19 = pd.DataFrame(sales_a_gen_dic_19)
sales_appstore_19 = sales_appstore_19.T
sales_appstore_19.columns = col
sales_appstore_19 = sales_appstore_19.drop([sales_appstore_19.index[1]])
sales_appstore_19 = sales_appstore_19.reset_index()
sales_appstore_19 = sales_appstore_19.set_index('index')
sales_appstore_19.head()

,19_1,19_2,19_3,19_4,19_5,19_6,19_7,19_8,19_9,19_10,19_11,19_12
index,,,,,,,,,,,,
NaN,0,0,0,0,0,0,0,0,0,0,0,0
자동차 경주,68,42,0,0,0,0,0,0,0,0,0,0
Social Networking,3,3,21,11,11,12,31,0,0,0,0,0
롤플레잉\n (일 경과),0,0,0,0,0,0,0,41,0,0,0,0
Role Playing,30,9,0,0,0,0,0,0,0,0,0,0


In [ ]:
sales_a_standard_19 = sales_appstore_19[['19_1','19_2','19_3','19_4','19_5','19_6','19_7','19_8','19_9']]
sales_a_standard_19 = sales_a_standard_19.reset_index()
sales_a_standard_19 = sales_a_standard_19.set_index('index')

In [ ]:
sales_a_standard_19['average'] = 0.000

for i in range(len(sales_a_standard_19)):
  total = 0
  for j in range(10):
    total += sales_a_standard_19.iloc[i][j]
  sales_a_standard_19['average'][i] = total/9

sales_a_standard_19 = sales_a_standard_19[['average']]
sales_a_standard_19 = sales_a_standard_19.reset_index()
genre = list(sales_a_standard_19['index'])
sales_a_standard_19 = sales_a_standard_19.set_index('index')
sales_a_standard_19.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,average
index,
NaN,0.000000
자동차 경주,12.222222
Social Networking,10.222222
롤플레잉\n (일 경과),4.555556
Role Playing,4.333333


In [ ]:
sales_appstore_19 = sales_appstore_19[['19_10','19_11','19_12']]

col = []
for i in range(10,13):
  col.append(str(i)+'월')

sales_appstore_19 = sales_appstore_19.reset_index()

for i in range(len(col)):
  sales_appstore_19[col[i]] = 0.000

for i in range(len(sales_appstore_19)):
  base = sales_appstore_19['index'][i]
  if base in genre:
    idx = genre.index(base)
    base = sales_a_standard_19.iloc[idx]['average']
  else:
    base = 1.0
  for j in range(2, 5):
    if base == 0:
      value = 0.0
    else:
      value = (float(sales_appstore_19.iloc[i][j]) - base) / base
    sales_appstore_19[col[j-2]][i] = value

sales_appstore_19 = sales_appstore_19.set_index('index')
sales_appstore_19 = sales_appstore_19[['10월','11월','12월']]
sales_appstore_19 = sales_appstore_19.reset_index()
sales_appstore_19.columns = ['genre', '10월','11월','12월']
month = list()
for i in range(10,13):
  month.append(list(sales_appstore_19[str(i)+'월']))
genre_list = list(sales_appstore_19['genre'])

sales_appstore_19_graph = pd.DataFrame()
for i in range(3):
  sales_appstore_19_fig = pd.DataFrame(genre_list,columns = ['genre'])
  sales_appstore_19_fig['value'] = month[i]
  sales_appstore_19_fig['month'] = str(i+10)+'월'
  sales_appstore_19_fig = sales_appstore_19_fig[['genre','month','value']]
  sales_appstore_19_graph = pd.concat([sales_appstore_19_graph,sales_appstore_19_fig], axis = 0)

sales_appstore_19_graph.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,genre,month,value
0,NaN,10월,0.0
1,자동차 경주,10월,-1.0
2,Social Networking,10월,-1.0
3,롤플레잉\n (일 경과),10월,-1.0
4,Role Playing,10월,-1.0


In [ ]:
appRank_sales_appstore_20_df.drop(['Unnamed: 0'], axis = 1, inplace=True)

tmp = [i for i in list(appRank_sales_appstore_20_df) if 'gen' in i]
sales_a_gen = list(set(np.array(appRank_sales_appstore_20_df[tmp]).reshape(len(tmp)*100,)))
sales_a_gen_dic_20 = []
for i in tmp:
  sales_a_gen_dic_tmp = {key: 0 for key in dict.fromkeys(sales_a_gen).keys()}
  score = 100
  for j in appRank_sales_appstore_20_df.dropna()[i]:
    sales_a_gen_dic_tmp[j] = sales_a_gen_dic_tmp[j]+score
    score -= 1
  sales_a_gen_dic_20.append(sales_a_gen_dic_tmp)

KeyError: ignored

In [ ]:
col = []
for i in range(6):
  col.append('20_'+str(i+1))

sales_appstore_20 = pd.DataFrame(sales_a_gen_dic_20)
sales_appstore_20 = sales_appstore_20.T
sales_appstore_20.columns = col
sales_appstore_20 = sales_appstore_20.drop([sales_appstore_20.index[1]])
sales_appstore_20 = sales_appstore_20.reset_index()

col = []
for i in range(1,7):
  col.append(str(i)+'월')

for i in range(len(col)):
  sales_appstore_20[col[i]] = 0.000

for i in range(len(sales_appstore_20)):
  base = sales_appstore_20['index'][i]
  if base in genre:
    idx = genre.index(base)
    base = sales_a_standard_19.iloc[idx]['average']
    if base == 0.0:
      base = 1.0
  else:
    base = 1.0
  for j in range(1, 7):
    if base == 0:
      value = 0.0
    else:
      value = (float(sales_appstore_20.iloc[i][j]) - base) / base
    sales_appstore_20[col[j-1]][i] = value

sales_appstore_20 = sales_appstore_20.set_index('index')
sales_appstore_20 = sales_appstore_20[['1월','2월','3월','4월','5월','6월']]
sales_appstore_20 = sales_appstore_20.reset_index()
sales_appstore_20.columns = ['genre', '1월','2월','3월','4월','5월','6월']
sales_appstore_20.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,genre,1월,2월,3월,4월,5월,6월
0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,자동차 경주,-1.000000,-1.000000,-1.000000,-1.000000,7.181818,7.181818
2,시뮬레이션,0.622677,1.258364,1.040892,-0.698885,0.923792,0.488848
3,교육,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,캐주얼 게임,0.107692,0.302797,-0.093706,0.120280,-0.383217,0.038462


In [ ]:
month = list()
for i in range(1,7):
  month.append(list(sales_appstore_20[str(i)+'월']))
genre_list = list(sales_appstore_20['genre'])

sales_appstore_20_graph = pd.DataFrame()
for i in range(6):
  sales_appstore_20_fig = pd.DataFrame(genre_list,columns = ['genre'])
  sales_appstore_20_fig['value'] = month[i]
  sales_appstore_20_fig['month'] = str(i+1)+'월'
  sales_appstore_20_fig = sales_appstore_20_fig[['genre','month','value']]
  sales_appstore_20_graph = pd.concat([sales_appstore_20_graph,sales_appstore_20_fig], axis = 0)

sales_appstore_20_graph.head()

,genre,month,value
0,NaN,1월,0.000000
1,자동차 경주,1월,-1.000000
2,시뮬레이션,1월,0.622677
3,교육,1월,0.000000
4,캐주얼 게임,1월,0.107692


In [ ]:
sales_appstore_19_20_graph = pd.concat([sales_appstore_19_graph, sales_appstore_20_graph], axis = 0)
df = sales_appstore_20_graph

fig = px.bar(df, x="genre", y="value", color="genre",
  animation_frame="month", animation_group="genre", range_y=[min(sales_appstore_19_20_graph['value'])-5, max(sales_appstore_19_20_graph['value'])+5])
fig.show()

KeyError: ignored

#### 결론 함께 내기!!!